# Assignment WE04-Universal Bank

Universal bank has recently trialed a marketing campaign to sell their new Securities account product to existing customers. They contacted 5000 of their non-Securities account customers with an offer. The data provided in universal.csv is the result of this market test. 

Use the techniques covered in this class to load and clean the data. Then, identify the best predictive model (using only the models covered thus far: Logistic Regression, SVM (with various kernels), and Decision trees). Your target variable is Securities Account. Your scoring measure is precision. Use RandomSearchCV combined with GridSearchCV to identify the best parameters for each model tested.

Be sure to document your thought process using markdown. Think of this as a report that your manager will read. This assignment requires you to decide how to process the provided data best (i.e., encoding). Be sure to provide your arguments/observations in markdown as you progress through data preparation, fitting, and performance evaluation.


    Id: Customer ID
    Age: Customers age in completed years  
    Experience: Number of years of professional experience  
    Income: Annual income of the customer ($000s)  
    Family Size: Family size of the customer  
    CCAvg: Average spending on credit cards per month ($000s)  
    Education: Education Level. 1: Undergrad; 2: Graduate; 3: Advanced/Professional  
    Mortgage: Value of house mortgage if any ($000s)  
    Personal Loan: (1 if customer has personal loand with bank, 0 otherwise)
    Securities Account: (1 if customer has securities account with bank, 0 otherwise)  
    CD Account: (1 if customer has certificate of deposit (CD) account with bank, 0 otherwise)  
    Online Banking: (1 if customer uses Internet banking facilities, 0 otherwise)  
    Credit Card: (1 if customer uses credit card issued by Universal Bank, 0 otherwise)

# 1.0 Import libraries and set random seed

In [15]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

np.random.seed(1)


## 2.0 Data loading

In [16]:
Ubank = pd.read_csv("./data/UniversalBank.csv")

## 3.0 Data cleaning
"Target variable is `Securities Account`"

### 3.1 Exploration of the data

In [17]:
Ubank.head(4)

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0


Quick info: missing data, how many observations, columns and names:

In [18]:
Ubank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


Generate a statistical summary of the numeric value in the data:

In [19]:
Ubank.describe()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,2500.500000,45.338400,20.104600,73.774200,93152.503000,2.396400,1.937938,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,1443.520003,11.463166,11.467954,46.033729,2121.852197,1.147663,1.747659,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,1.000000,23.000000,-3.000000,8.000000,9307.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1250.750000,35.000000,10.000000,39.000000,91911.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,2500.500000,45.000000,20.000000,64.000000,93437.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,3750.250000,55.000000,30.000000,98.000000,94608.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,5000.000000,67.000000,43.000000,224.000000,96651.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


**Observations:**
* There are no missing data.
* I agree with the encoding of the categorical variables. 
* Drop ID feature since is not relevant for the prediction.
* There is at least one ZIP code wrong since it has 4 digits. Therefore they can be dropped or imputed (after the data splitting). It depends on how many there are.
* The Experience feature must not have negative values. Therefore those values can be droppend or imputed using the mean of the variable after the data spltting.

In [20]:
#drop ID feature
Ubank.drop('ID', axis=1, inplace = True)

In [21]:
#check how many ZIP codes are wrong
zipdigits = np.int_(np.log10(np.array(Ubank['ZIP Code'])) + 1)
zip_index = list(np.where(zipdigits != 5)[0])
print(len(zip_index))
print(Ubank.loc[zip_index,['ZIP Code']])

1
     ZIP Code
384      9307


In [22]:
#check how many Experience observations are negative
experience_index = list(np.where(np.array(Ubank['Experience']) < 0)[0])
len(experience_index)

52

In [23]:
# Drop wrong ZIP codes and Experience values
Ubank.drop(zip_index, axis=0, inplace = True)
Ubank.drop(experience_index, axis=0, inplace = True)
Ubank.describe()

,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
count,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000,4947.000000
mean,45.556095,20.330099,73.825147,93168.521932,2.391146,1.936196,1.878714,56.645846,0.097029,0.104306,0.061047,0.596927,0.293916
std,11.321615,11.312922,46.111141,1761.253907,1.148333,1.747768,0.839679,101.835994,0.296026,0.305688,0.239441,0.490565,0.455600
min,24.000000,0.000000,8.000000,90005.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,36.000000,10.500000,39.000000,91911.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,46.000000,20.000000,64.000000,93437.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,55.000000,30.000000,98.000000,94608.000000,3.000000,2.600000,3.000000,101.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,67.000000,43.000000,224.000000,96651.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### 3.2 Split data (train/test)

In [24]:
# split the data into validation and training set
train_df, test_df = train_test_split(Ubank, test_size=0.3)

# create variables to represent the columns
# that are our predictors and target
target = 'Securities Account'
predictors = list(Ubank.columns)
predictors.remove(target)

### 3.3  Conduct any data prepartion that should be done *AFTER* the data split


Remove differences of scale by **standardizing** the numerical variables.

In [25]:
# create a standard scaler and fit it to the training set of predictors
scaler = preprocessing.StandardScaler()
cols_to_stdize = ['Age', 'Experience', 'Income', 
                   'ZIP Code', 'Family', 'CCAvg', 'Mortgage']  
               
               
# Transform the predictors of training and validation sets
train_df[cols_to_stdize] = scaler.fit_transform(train_df[cols_to_stdize]) # train_predictors is not a numpy array

test_df[cols_to_stdize] = scaler.transform(test_df[cols_to_stdize]) # validation_target is now a series object


In [26]:
train_df.describe()

,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
count,3.462000e+03,3.462000e+03,3.462000e+03,3.462000e+03,3.462000e+03,3.462000e+03,3462.000000,3.462000e+03,3462.000000,3462.000000,3462.000000,3462.000000,3462.000000
mean,1.571373e-17,-7.946657e-17,-1.478534e-16,1.266366e-15,-2.543700e-16,1.390184e-17,1.879549,3.897005e-16,0.102542,0.107163,0.058637,0.599942,0.297805
std,1.000144e+00,1.000144e+00,1.000144e+00,1.000144e+00,1.000144e+00,1.000144e+00,0.838053,1.000144e+00,0.303403,0.309366,0.234977,0.489981,0.457359
min,-1.885039e+00,-1.779143e+00,-1.423802e+00,-1.770340e+00,-1.218872e+00,-1.108785e+00,1.000000,-5.537520e-01,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-9.173502e-01,-8.983740e-01,-7.568999e-01,-7.718598e-01,-1.218872e+00,-7.113473e-01,1.000000,-5.537520e-01,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-3.763319e-02,-1.760520e-02,-2.190756e-01,1.487727e-01,-3.454017e-01,-2.571325e-01,2.000000,-5.537520e-01,0.000000,0.000000,0.000000,1.000000,0.000000
75%,8.420838e-01,8.631636e-01,5.553914e-01,8.262724e-01,5.280685e-01,3.674128e-01,3.000000,4.259143e-01,0.000000,0.000000,0.000000,1.000000,1.000000
max,1.897744e+00,1.920086e+00,3.093922e+00,1.978755e+00,1.401539e+00,4.568900e+00,3.000000,5.667130e+00,1.000000,1.000000,1.000000,1.000000,1.000000


## 4.0 Save the data

In [27]:
train_X = train_df[predictors]
train_y = train_df[target] 
test_X = test_df[predictors]
test_y = test_df[target] 

train_df.to_csv('./data/ubank_train_df.csv', index=False)
train_X.to_csv('./data/ubank_train_X.csv', index=False)
train_y.to_csv('./data/ubank_train_y.csv', index=False)

test_df.to_csv('./data/ubank_test_df.csv', index=False)
test_X.to_csv('./data/ubank_test_X.csv', index=False)
test_y.to_csv('./data/ubank_test_y.csv', index=False)